In [5]:
%pip install pycryptodome

In [ ]:
# ============================================================
# Ejemplo de criptografía asimétrica RSA en Python
# ============================================================
# Conceptos aplicados:
# - Cifrado asimétrico (dos claves: pública y privada)
# - Padding OAEP (para cifrado seguro)
# - Padding PSS (para firma digital segura)
# - Librería: PyCryptodome
# ============================================================

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pss
from Crypto.Hash import SHA256
import base64

# 1️. Generar el par de claves (privada y pública)
# ------------------------------------------------
# La clave privada incluye la pública; ambas se usan para distintos propósitos.
key = RSA.generate(2048)  # 2048 bits = longitud estándar segura
private_key = key
public_key = key.publickey()

print("🔐 Clave pública (exportada):")
print(public_key.export_key().decode())
print()

# 2️. Cifrar un mensaje con la clave pública
# ------------------------------------------------
# Solo quien tenga la clave privada podrá descifrarlo.
mensaje = "Hola, este es un mensaje cifrado con RSA + OAEP"
print(f"📩 Mensaje original: {mensaje}")

# Creamos el cifrador con padding OAEP (seguro)
cipher_rsa = PKCS1_OAEP.new(public_key, hashAlgo=SHA256)
mensaje_cifrado = cipher_rsa.encrypt(mensaje.encode())

print("\n🔒 Mensaje cifrado (base64):")
print(base64.b64encode(mensaje_cifrado).decode())

# 3️. Descifrar con la clave privada
# ------------------------------------------------
cipher_rsa_priv = PKCS1_OAEP.new(private_key, hashAlgo=SHA256)
mensaje_descifrado = cipher_rsa_priv.decrypt(mensaje_cifrado)

print("\n🔓 Mensaje descifrado:")
print(mensaje_descifrado.decode())

# 4️. Firmar un mensaje con la clave privada (PSS)
# ------------------------------------------------
# El emisor usa su clave privada para crear una firma digital única del mensaje.
hash_mensaje = SHA256.new(mensaje.encode())
signature = pss.new(private_key).sign(hash_mensaje)

print("\n✍️ Firma digital (base64):")
print(base64.b64encode(signature).decode())

# 5️. Verificar la firma con la clave pública
# ------------------------------------------------
# El receptor usa la clave pública del emisor para validar la firma.
verificador = pss.new(public_key)

# ================================================
#  SECCIÓN OPCIONAL PARA PROBAR FIRMA INVÁLIDA
# -----------------------------------------------
# Si se descomenta UNA de estas líneas, la verificación fallará:
# (simula manipulación o error en la transmisión)
# ================================================

# 1️. Cambiar el mensaje (simula alteración del contenido)
#  hash_mensaje = SHA256.new("Mensaje modificado".encode())

# 2️. Corromper la firma (simula manipulación de la firma)
# signature = b"\x00" + signature[1:]

# 3️. Usar otra clave pública (simula atacante o identidad falsa)
# otro_key = RSA.generate(2048)
# public_key = otro_key.publickey()
# verificador = pss.new(public_key)

# ================================================

try:
    verificador.verify(hash_mensaje, signature)
    print("\n✅ Firma verificada: El mensaje proviene del dueño de la clave privada.")
except (ValueError, TypeError):
    print("\n❌ Firma inválida o el mensaje fue modificado.")


🔐 Clave pública (exportada):
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAndhmgUzd+j3jOPwBjgLA
2GoX+E7M051EwKkOltq1mHKSxsHCMTJKsWaunrRFcztyG8NEMvVqz5CmLTIZpgUe
axr5XeNc7XLOM994hdWqmi0XHULEBPWysvpu4KTFoXJPkJxzLxK6efcq2UmU3W64
/VuoaHAxyot9b203HO5539YxoNpfGaGBenITo1oYWPprsP/H27YOBuGVxNKHPsJ3
p83oJpA+aL03tDCkkUfeqJWAtovmiqnq8QIbEkLnxiZtPY7G525d5im3O1XTiT7o
idMMiot5wRDC5ftF1l33CC0kc/F/kRR0c9e1uqMbnyRPcwpTSleg+uPz6OlQ5JkR
GQIDAQAB
-----END PUBLIC KEY-----

📩 Mensaje original: Hola, este es un mensaje cifrado con RSA + OAEP

🔒 Mensaje cifrado (base64):
Jeaoh16e+OMnwJZA86WG4nwWi8B3d0fTmbJTOfLR4Ob/UB5s7t4cZ+nKQDt9eZ99+a4NLcoCsFeSiOOITu23GoZbnwFvFAgJCRMMQUCEsr6URdOQL8bfZBmVPWSnAo+MNi5JHK8oXg+36c5RG5nlhYX44asaUgo0vOUfutF6RUMI/HSoE0Lu8fa9r/Vp7Be73JzMdfpQ65Ym9eY8eQlSMQCAXn4DDIh4D7dBHtGPwYhF01jCWOXeupj5ahkHNU0crBuZUWEx5e5j6fgHyodQ9Osm6yHj4In5NZuLJHzi2yWbHvdmp3N6Gis5y5zeDwWm5IuZ6Z2oXh6DvcTHzP5F0g==

🔓 Mensaje descifrado:
Hola, este es un mensaje cifrado con RSA + OAEP

✍️ Fi